In [1]:
import os
import torch
import random
import math
from torch import nn
from torch.nn.modules import activation
import torch.nn.functional as F
import torchvision
import shutil
import numpy as np
from PIL import Image
import time
from tqdm.notebook import tqdm
from sklearn.metrics import confusion_matrix

C:\Users\GillA\Anaconda3\envs\Aseem\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
class MSCB(nn.Module):
    def __init__(self, small_kernel, medium_kernel, large_kernel, num_filters):
        super(MSCB, self).__init__()
        self.name = "MSCB"

        # Define Small Path
        self.convS = nn.Conv1d(in_channels = 56, out_channels = num_filters, kernel_size = small_kernel, padding = 'same')
        self.MPoolS = nn.MaxPool1d(kernel_size = small_kernel, stride = 5, padding = int(small_kernel/2 - 1))
        
        # Define Medium Path
        self.convM = nn.Conv1d(in_channels = 56, out_channels = num_filters, kernel_size = medium_kernel, padding = 'same')
        self.MPoolM = nn.MaxPool1d(kernel_size = medium_kernel, stride = 5, padding = int(medium_kernel/2 - 1))
        
        # Define Large Path
        self.convL = nn.Conv1d(in_channels = 56, out_channels = num_filters, kernel_size = large_kernel, padding = 'same')
        self.MPoolL = nn.MaxPool1d(kernel_size = large_kernel, stride = 5, padding = int(large_kernel/2 - 1))
        
        #
        self.MPool = nn.MaxPool1d(kernel_size = 3, stride = 5)
        
        #
        self.auxMPool = nn.MaxPool1d(kernel_size = 10, stride = 10)
#         self.avgpool = nn.AdaptiveAvgPool1d(12)
        
        self.auxFC = nn.Linear(in_features = 9600, out_features = 3840)
        
        self.conv = nn.Conv1d(in_channels = 56, out_channels = 128, kernel_size = 24, padding = 'same')
        
        #
        self.conv2 = nn.Conv1d(in_channels = 3*num_filters + 128, out_channels = 64, kernel_size = 112, padding = 'same')
        self.MPool2 = nn.MaxPool1d(kernel_size = 3, stride = 5)
        self.conv3 = nn.Conv1d(in_channels = 64, out_channels = 64, kernel_size = 25, padding = 'same')
        self.fc1 = nn.Linear(in_features = 3840, out_features = 400)
        self.Dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(in_features = 400, out_features = 1024)
        self.fc3 = nn.Linear(in_features = 1024, out_features = 3)

    def forward(self, x):
        ### Feature Learning Head
        
        x = torch.moveaxis(x,2,1)
        x = torch.tensor(x, dtype=torch.float32)

        # Parrallel Convolutions
        x_S = self.MPoolS(F.relu(self.convS(x)))
        x_M = self.MPoolM(F.relu(self.convM(x)))
        x_L = self.MPoolL(F.relu(self.convL(x)))
        x_O = F.relu(self.conv(self.MPool(x)))
        
        # Concatenate
        x = torch.cat((x_S,x_M,x_L,x_O),1)
        
        ### Auxillary Classification
        x_aux = self.auxMPool(x)
        x_aux = x_aux.view(-1,9600)
        x_aux = self.auxFC(x_aux)
        # Classification Head
        x_aux = F.relu(self.fc1((x_aux)))
        x_aux = self.Dropout(x_aux)
        x_aux = F.relu(self.fc2(x_aux))
        x_aux = self.fc3(x_aux)
        
        # Back to Main
        x = self.MPool2(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        
        # Flattening
        x = x.view(-1,3840)

        #Classification Head
        x = F.relu(self.fc1((x)))
        x = self.Dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return (x, x_aux)



In [3]:
n_filters = 64
small_kernel = 1
medium_kernel = 3
large_kernel = 5
one_CNN = MSCB(small_kernel, medium_kernel, large_kernel, n_filters)
pytorch_total_params = sum(p.numel() for p in one_CNN.parameters())
print(pytorch_total_params)

41418835
